In [1]:
%reload_ext autoreload
%autoreload 2
import os
from pyeed.core import ProteinRecord

##### Here we get all the data

In [2]:
# read in the text file
file_path = 'TEM_Ids.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()

# create a dic with the name and id
# A	TEM-P118	t		AAN05029 	AY130285 	12354869 	view			2be	ESBL	A
# we want to keep the name and the id, these are the second and the fourth columns
# we need to split the line by the tab character
data = {}
count = 0
for line in lines:
    line = line.strip()
    columns = line.split('\t')
    name = columns[1]
    id = columns[4].strip()
    if id == 'EFJ83682':
        continue
    data[name] = id
    count += 1


print('EFJ83682' in list(data.values()))

False


In [3]:
protein_ids_fetched = ProteinRecord.get_ids(list(data.values()))

# here we want to add the region for the ncbi blast id
# this is a kind of work arround to get the id later for the name mappping
for i in range(0, len(protein_ids_fetched)):
    protein_ids_fetched[i].add_to_regions(
        name="blastquery:" + list(data.keys())[i],
        start=0,
        end=len(protein_ids_fetched[i].sequence)
    )

print(protein_ids_fetched[0])

Output()

ProteinRecord
├── id = APG33178.1
├── name = class A extended-spectrum beta-lactamase TEM-9
├── organism
│   └── Organism
│       ├── id = 562
│       ├── taxonomy_id = 562
│       └── name = Escherichia coli
├── sequence = MSIQHFRVALIPFFAAFCFPVFAHPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMSTFKVLLCGAVLSRVDAGQEQLGRRIHYSQNDLVKYSPVTEKHLTDGMTVRELCSAAITMSDNTAANLLLTTIGGPKELTAFLHNMGDHVTRLDSWEPELNEAIPNDERDTTMPAAMATTLRKLLTGELLTLASRQQLIDWMEADKVAGPLLRSALPAGWFIADKSGAGERGSRGIIAALGPDGKPSRIVVIYMTGSQATMDERNRQIAEIGASLIKHW
├── regions
│   ├── 0
│   │   └── Region
│   │       ├── id = PRK15442
│   │       ├── start = 2
│   │       └── end = 286
│   └── 1
│       └── Region
│           ├── id = c64dba5a-5b47-4033-aeb0-a5df9fb22667
│           ├── name = blastquery:TEM-1
│           ├── start = 0
│           └── end = 286
└── coding_sequence
    └── 0
        └── Region
            ├── id = KY271103.1
            ├── start = 1
            └── end = 861



In [4]:
current_path = os.path.dirname(os.getcwd())
print(len(protein_ids_fetched))

210


In [5]:
# we now search for each protein in the data dic a blast
# then we combine the results im data, we also have to remove double entries
data_proteins = []

"""
for protein in protein_ids_fetched:
    data_proteins = data_proteins + protein.ncbi_blast(n_hits=1000, e_value=0.1)
"""
"""
# no i want to parallelize this
import asyncio

async def fetch_protein(protein):
    return protein.ncbi_blast(n_hits=1000, e_value=0.1)

# create a job for each protein
jobs = [fetch_protein(protein) for protein in protein_ids_fetched]
# run all the jobs
data_proteins = await asyncio.gather(*jobs)
"""

from concurrent.futures import ThreadPoolExecutor  # noqa: E402

def fetch_protein(protein):
    return protein.ncbi_blast(n_hits=150, e_value=0.1)


with ThreadPoolExecutor(max_workers=15) as executor:
    results = list(executor.map(fetch_protein, protein_ids_fetched))


Output()

Output()

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

Output()

In [ ]:
# add the protein_ids_fetched to results
for protein in protein_ids_fetched:
    results.append(protein)

data_proteins = results
# flatten the list
data_proteins = [item for sublist in data_proteins for item in sublist]



In [ ]:
# remove double entries
data_protein_unique = []
data_protein_unique_id = []
for protein in data_proteins:
    if type(protein) != ProteinRecord:
        print(protein)
        continue
    if protein.id not in data_protein_unique_id:
        data_protein_unique.append(protein)
        data_protein_unique_id.append(protein.id)

print(len(data_protein_unique))


('id', 'AJO16045.1')
('name', 'class A beta-lactamase blaTEM-215')
('organism', Organism(id='562', taxonomy_id=562, name='Escherichia coli', domain=None, kingdom=None, phylum=None, tax_class=None, order=None, family=None, genus=None, species=None, json_ld_type=['Organism'], json_ld_context={'Organism': 'https://github.com/PyEED/pyeed/Organism', 'taxonomy_id': 'http://edamontology.org/data_1179', 'name': 'http://edamontology.org/data_2909', 'kingdom': 'http://edamontology.org/data_1044', 'family': 'http://edamontology.org/data_2732', 'genus': 'http://edamontology.org/data_1870', 'species': 'http://edamontology.org/data_1045'}))
('sequence', 'MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMSTFKVLLCGAVLSRVDAGQEQLGRRIHYSQNDLVEYSPVTEKHLTDGMTVRELCSAAITMSDNTAANLLLTTIGGPKELTAFLRNMGDHVTRLDRWEPELNEAIPNDERDTTMPAAMATTLRKLLTGELLTLASRQQLIDWMEADKVAGPLLRSALPAGWFIADKSGAGERGSRGIIAALGPDGKPSRIVVIYTTGSQATMDERNRQIAEIGASLIKHW')
('seq_length', None)
('sites', [])
('regions', [Region(id='PRK

In [ ]:
# save the blast search results
output_folder_blast_search = os.path.join(current_path, "TEM-lactamase", "data", "data_blast_search_swissprot_all_ids")
os.makedirs(output_folder_blast_search, exist_ok=True)
counter = 0
for hit in data_proteins:
    with open(output_folder_blast_search + "/{}.json".format(counter), "w") as f:
        f.write(hit.json())
        counter += 1